In [51]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error


In [52]:
df = pd.read_csv('hour.csv')

In [53]:
df_clean = df.drop(["instant", "yr", "dteday","casual","registered"],axis=1) 
df_clean.describe() 

,season,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,2.501640,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,189.463088
std,1.106918,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,181.387599
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,40.000000
50%,3.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,142.000000
75%,3.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,281.000000
max,4.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,977.000000


In [54]:
#df_clean = df_clean.groupby(['season','mnth','hr','holiday','weekday','workingday','weathersit']).agg({'temp': ['mean'],'temp': ['mean'],'atemp': ['mean'], 'hum': ['mean'], 'windspeed': ['mean'],'cnt': ['mean']}).reset_index()
# df_clean = df_clean.groupby(['season','mnth','hr','holiday','weekday','workingday','weathersit','temp','atemp','hum','windspeed']).agg({'cnt': ['mean']}).reset_index()

df_clean = df_clean.groupby(['season','mnth','hr','holiday','weekday','workingday','weathersit']).agg({'temp': ['mean'],'temp': ['mean'],'atemp': ['mean'], 'hum': ['mean'], 'windspeed': ['mean'],'cnt': ['mean']}).reset_index()
X=df_clean.drop(columns="cnt",axis=1)
y=df_clean['cnt']


/tmp/ipykernel_16325/2423819378.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X=df_clean.drop(columns="cnt",axis=1)


In [55]:
#X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2,random_state=2)

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)
X_train,X_test,y_train,y_test=train_test_split(scaled_X, y, test_size=0.2, random_state=2)

In [56]:
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# 5. BackPropagation Multi Layer Perceptron regressor with tree hidden layer
mlp = MLPRegressor()
parameters = {
    'hidden_layer_sizes': [(64,64,64), (96,96,96), (128,128,128), (196,196,196)],
    'activation': ['tanh'],
    'solver': ['sgd'],
    'max_iter': [7000, 8000, 10000],
    'learning_rate_init': [0.0001, 0.0005, 0.001]
}
grid_search = GridSearchCV(mlp, parameters, n_jobs=-1, cv=2)
grid_search.fit(X_train, np.ravel(y_train, order="C"))
y_pred = grid_search.predict(X_test)

rscore = grid_search.score(X_test, y_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

In [57]:
print(grid_search.best_params_)
print(grid_search.best_estimator_)

print("Score: %.4f" % rscore)
print("MSE: %.4f" % mse)
print("MAE: %.4f" % mae)
print("MAPE: %.2f" % mape)

{'activation': 'tanh', 'hidden_layer_sizes': (196, 196, 196), 'learning_rate_init': 0.0001, 'max_iter': 8000, 'solver': 'sgd'}
MLPRegressor(activation='tanh', hidden_layer_sizes=(196, 196, 196),
             learning_rate_init=0.0001, max_iter=8000, solver='sgd')
Score: 0.9100
MSE: 2346.6892
MAE: 30.2056
MAPE: 0.35


In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel("Actual values")
plt.ylabel("Predicted values")
plt.title("Actual vs Predicted values")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.show()

In [59]:
mlp = MLPRegressor(activation='tanh', hidden_layer_sizes=(196, 196, 196), learning_rate_init=0.0001, max_iter=8000, solver='sgd', random_state=2)
mlp.fit(X_train, np.ravel(y_train, order="C"))
y_pred = mlp.predict(X_test)

rscore = mlp.score(X_test, y_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Score: %.4f" % rscore)
print("MSE: %.4f" % mse)
print("MAE: %.4f" % mae)
print("MAPE: %.2f" % mape)

Score: 0.9054
MSE: 2466.3722
MAE: 30.3335
MAPE: 0.36


# Conclusions
- Best model based on R2 score is: 5. BackPropagation Multi Layer Perceptron regressor with tree hidden layer
- The model with Back Propagation improved 0.913% over the second best model (3)
- Other metrics in model 5) also improved, like MAPE that reduced from 0.55 to 0.37, compared to the second best model (3)

In [60]:
from sklearn.pipeline import Pipeline
import joblib
pipeline = Pipeline([('scaler', scaler), ('regressor', mlp)])

# save the model to disk
joblib.dump(pipeline, 'bikeshare.pkl')

['bikeshare.pkl']